In [116]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.statespace.sarimax import SARIMAX
from scipy.stats import norm


In [117]:
# Load files

train_data = pd.read_csv('regression_input_data.csv')

#temp solution while haven't downloaded from correct source
temp = pd.read_csv('temp_proj_gville_county.csv')
pre = pd.read_csv('pre_proj_gville_county.csv')
pop = pd.read_csv('pop_proj_gville_county.csv')



In [118]:
# Define the year range
y_low = 2019
y_upper = 2023
start_year = 2019
end_year = 2023

In [119]:
#preprocess prec data


#precipitation
pre = pre.drop(columns=["ANNUAL"])

#Filter precipitation for the time range
pre['YEAR'] = pd.to_numeric(pre['YEAR'], errors='coerce')

pre = pre[(pre['YEAR'] >= start_year) & (pre['YEAR'] <= end_year)] 
#pre.set_index('YEAR', inplace=True)

# Reshape the data from wide to long format
pre_long = pre.melt(id_vars=["YEAR"], var_name="month", value_name="pre")


# Map month columns (M1, M2, ...) to actual month names
month_map = {
    "M1": "Jan", "M2": "Feb", "M3": "Mar", "M4": "Apr", "M5": "May",
    "M6": "Jun", "M7": "Jul", "M8": "Aug", "M9": "Sep", "M10": "Oct",
    "M11": "Nov", "M12": "Dec"
}
pre_long["month"] = pre_long["month"].map(month_map)



# Filter out rows where 'pre' is "M"
pre_long = pre_long[pre_long["pre"] != "M"]

# Rename columns for clarity
pre_long = pre_long.rename(columns={"YEAR": "year"})

# Reset index for a clean output
pre_long = pre_long.reset_index(drop=True)
pre = pre_long


#display(pre)



In [120]:
#preprocess population
# Interpolate population from the 'pop' sheet

# Drop the 'COUNTY' column
pop = pop.drop(columns=['COUNTY'])  # Drop the 'COUNTY' column

# Assuming pop is your DataFrame
pop.columns = ['year', 'pop']  # Rename columns to 'year' and 'pop'

pop = pop[(pop['year'] >= start_year) & (pop['year'] <= end_year)]

pop.reset_index(drop=True, inplace=True) 

rows = []

# Create a DataFrame with all months
result = pd.DataFrame(columns=['year', 'month', 'pop'])

# Iterate through each year and interpolate monthly data
for i in range(len(pop) - 1):
    s_year = pop.loc[i, 'year']
    e_year = pop.loc[i + 1, 'year']
    start_pop = pop.loc[i, 'pop']
    end_pop = pop.loc[i + 1, 'pop']
    
    # Generate monthly data
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    monthly_pops = np.linspace(start_pop, end_pop, 12, endpoint=False)
    monthly_pops = np.round(monthly_pops).astype(int)  # Round to whole numbers
    
    # Append to result DataFrame
    for j, month in enumerate(months):
        rows.append({'year': s_year, 'month': month, 'pop': monthly_pops[j]})
        
# Add data for the final year
last_year = pop.loc[len(pop) - 1, 'year']
last_pop = pop.loc[len(pop) - 1, 'pop']
for month in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']:
        rows.append({'year': last_year, 'month': month, 'pop': last_pop})
        
result = pd.DataFrame(rows)

pop = result
pop = pop.reset_index(drop=True)

#display(pop)

In [121]:
# Preprocess temperature data

temp = temp.drop(columns=["ANNUAL"])

#Filter precipitation for the time range
temp['YEAR'] = pd.to_numeric(temp['YEAR'], errors='coerce')

temp = temp[(temp['YEAR'] >= start_year) & (temp['YEAR'] <= end_year)] 
#pre.set_index('YEAR', inplace=True)

# Reshape the data from wide to long format
temp_long = temp.melt(id_vars=["YEAR"], var_name="month", value_name="temp")


# Map month columns (M1, M2, ...) to actual month names
month_map = {
    "M1": "Jan", "M2": "Feb", "M3": "Mar", "M4": "Apr", "M5": "May",
    "M6": "Jun", "M7": "Jul", "M8": "Aug", "M9": "Sep", "M10": "Oct",
    "M11": "Nov", "M12": "Dec"
}
temp_long["month"] = temp_long["month"].map(month_map)



# Filter out rows where 'pre' is "M"
temp_long = temp_long[temp_long["temp"] != "M"]

# Rename columns for clarity
temp_long = temp_long.rename(columns={"YEAR": "year"})

# Reset index for a clean output
temp_long = temp_long.reset_index(drop=True)
temp = temp_long

#display(temp)

In [122]:
# Merge all data into one df


# Merge all DataFrames based on 'year' and 'month'

prediction_data = pop.merge(temp, on=['year', 'month'], how='outer')  # Merge pop and temp
prediction_data = prediction_data.merge(pre, on=['year', 'month'], how='outer')  # Merge with precip


#  Fill  values with blank (empty) values
prediction_data['pop'] = prediction_data['pop'].replace(np.nan, '')
prediction_data['pre'] = prediction_data['pre'].replace(np.nan, '')
prediction_data['temp'] = prediction_data['temp'].replace(np.nan, '')

month_map = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}

# Add a new column to convert month abbreviations to numeric values
prediction_data['month_num'] = prediction_data['month'].map(month_map)

# Create month indicator variables (dummy variables for months)
month_dummies = pd.get_dummies(prediction_data['month_num'], prefix='M')

# Convert Boolean True/False to 1/0
month_dummies = month_dummies.astype(int)

# Ensure the dummy columns are named 'M1' to 'M12'
month_dummies.columns = [f'M{i}' for i in range(1, 13)]

# Merge the month indicator variables with your train_data
prediction_data = pd.concat([prediction_data, month_dummies], axis=1)

prediction_data = prediction_data.drop(columns=['month_num'])

display(prediction_data)


,year,month,pop,temp,pre,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,M11,M12
0,2019,Jan,524748,43.7,5.53,1,0,0,0,0,0,0,0,0,0,0,0
1,2019,Feb,524956,50.4,6.72,0,1,0,0,0,0,0,0,0,0,0,0
2,2019,Mar,525165,51.4,2.87,0,0,1,0,0,0,0,0,0,0,0,0
3,2019,Apr,525373,62.9,4.15,0,0,0,1,0,0,0,0,0,0,0,0
4,2019,May,525582,74.1,1.77,0,0,0,0,1,0,0,0,0,0,0,0
5,2019,Jun,525790,75.9,4.77,0,0,0,0,0,1,0,0,0,0,0,0
6,2019,Jul,525998,80.2,3.73,0,0,0,0,0,0,1,0,0,0,0,0
7,2019,Aug,526207,78.5,6.76,0,0,0,0,0,0,0,1,0,0,0,0
8,2019,Sep,526415,79.1,0.18,0,0,0,0,0,0,0,0,1,0,0,0
9,2019,Oct,526624,66.5,4.98,0,0,0,0,0,0,0,0,0,1,0,0


In [123]:
# preprocess hist data

month_map = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}

# Add a new column to convert month abbreviations to numeric values
train_data['month_num'] = train_data['month'].map(month_map)

# Create month indicator variables (dummy variables for months)
month_dummies = pd.get_dummies(train_data['month_num'], prefix='M')

# Convert Boolean True/False to 1/0
month_dummies = month_dummies.astype(int)

# Ensure the dummy columns are named 'M1' to 'M12'
month_dummies.columns = [f'M{i}' for i in range(1, 13)]

# Merge the month indicator variables with your train_data
train_data = pd.concat([train_data, month_dummies], axis=1)

train_data = train_data.drop(columns=['month_num'])

display(train_data)

,year,month,gpcd,pop,pre,temp,gpd,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,M11,M12
0,1985,Jan,503.009104,302514,4.94,36.2,152167296,1,0,0,0,0,0,0,0,0,0,0,0
1,1985,Feb,311.351553,302697,4.29,44.6,94245181,0,1,0,0,0,0,0,0,0,0,0,0
2,1985,Mar,0.000000,302880,1.13,54.2,0,0,0,1,0,0,0,0,0,0,0,0,0
3,1985,Apr,0.000000,303063,1.31,61.8,0,0,0,0,1,0,0,0,0,0,0,0,0
4,1985,May,0.000000,303246,2.42,67.9,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,2015,Aug,0.000000,491218,2.41,79.1,0,0,0,0,0,0,0,0,1,0,0,0,0
368,2015,Sep,0.000000,491218,5.56,72.8,0,0,0,0,0,0,0,0,0,1,0,0,0
369,2015,Oct,22.312776,491218,9.44,60.2,10960437,0,0,0,0,0,0,0,0,0,1,0,0
370,2015,Nov,69.380739,491218,9.13,55.4,34081068,0,0,0,0,0,0,0,0,0,0,1,0


In [124]:

# Convert month names into one-hot encoded columns (M1-M12) for both train and prediction data
train_data = pd.concat([train_data, pd.get_dummies(train_data['month'], prefix='M')], axis=1)
prediction_data = pd.concat([prediction_data, pd.get_dummies(prediction_data['month'], prefix='M')], axis=1)

# Prepare the exogenous variables (X) for SARIMAX model
X_train = train_data[['temp', 'pre', 'year'] + [f'M{i}' for i in range(1, 13)]]
y_train = train_data['gpcd']

# Lower-GPCD fit: GLS with SARIMA(1, 0, 0) × (1, 0, 0)_11 errors
arima_model_lower = SARIMAX(y_train, exog=X_train, order=(1, 0, 0), seasonal_order=(1, 0, 0, 12), trend=None)
arima_result_lower = arima_model_lower.fit()

# Extract coefficients and p-values
lower_coefficients = arima_result_lower.params
lower_pval = 2 * (1 - norm.cdf(np.abs(lower_coefficients) / np.sqrt(np.diag(arima_result_lower.cov_params()))))

# Save the coefficients table for lower-GPCD
lower_coefficients_table = pd.DataFrame({'Coefficient': lower_coefficients, 'P-value': lower_pval})
lower_coefficients_table.to_csv('lower_coefficients_table.csv', sep='\t', index=False)

# Higher-GPCD: GLS with SARIMA(1, 0, 0) × (1, 0, 0)_12 errors
train_data['Byear'] = train_data['year']
train_data['Byear'] = np.where(train_data['Byear'] <= y_low, y_low, train_data['Byear'])
train_data['Byear'] = np.where(train_data['Byear'] >= y_upper, y_upper, train_data['Byear'])

X_train_byear = train_data[['temp', 'pre', 'year', 'Byear'] + [f'M{i}' for i in range(1, 13)]]
arima_model_byear = SARIMAX(y_train, exog=X_train_byear, order=(1, 0, 0), seasonal_order=(1, 0, 0, 12), trend=None)
arima_result_byear = arima_model_byear.fit()

# Extract coefficients and p-values
higher_coefficients = arima_result_byear.params
higher_pval = 2 * (1 - norm.cdf(np.abs(higher_coefficients) / np.sqrt(np.diag(arima_result_byear.cov_params()))))

# Save the coefficients table for higher-GPCD
higher_coefficients_table = pd.DataFrame({'Coefficient': higher_coefficients, 'P-value': higher_pval})
higher_coefficients_table.to_csv('higher_coefficients_table.csv', sep='\t', index=False)

# Predict GPCD for 24 different temperature and precipitation pairs
GPCD_year_predict = []
GPCD_byear_predict = []

# Adjust prediction years
prediction_data['Byear'] = prediction_data['year']
prediction_data['Byear'] = np.where(prediction_data['Byear'] <= y_low, y_low, prediction_data['Byear'])
prediction_data['Byear'] = np.where(prediction_data['Byear'] >= y_upper, y_upper, prediction_data['Byear'])

# Loop through all months and predict GPCD
for i in range(1, 25):
    # Get the temperature and precipitation columns for each pair
    tem_col = f'Tem{i}'
    pre_col = f'Pre{i}'

    # Set up the exogenous variables for prediction (same as for training)
    X_predict = prediction_data[['temp', 'pre', 'year'] + [f'M{i}' for i in range(1, 13)]]
    X_predict[tem_col] = prediction_data[tem_col]
    X_predict[pre_col] = prediction_data[pre_col]

    # Prediction with lower-GPCD model
    lower_prediction = arima_result_lower.get_prediction(start=0, end=len(prediction_data)-1, exog=X_predict)
    GPCD_year_predict.append(lower_prediction.predicted_mean.values)

    # Prediction with higher-GPCD model
    X_predict_byear = X_predict.copy()
    X_predict_byear['Byear'] = prediction_data['Byear']
    higher_prediction = arima_result_byear.get_prediction(start=0, end=len(prediction_data)-1, exog=X_predict_byear)
    GPCD_byear_predict.append(higher_prediction.predicted_mean.values)

# Convert the predictions into DataFrame format
GPCD_year_predict_df = pd.DataFrame(np.array(GPCD_year_predict).T)
GPCD_byear_predict_df = pd.DataFrame(np.array(GPCD_byear_predict).T)



KeyError: 'Tem1'

In [ ]:
# Save prediction results
GPCD_year_predict_df.to_csv('GPCD_year_predict.csv', sep='\t', index=False)
GPCD_byear_predict_df.to_csv('GPCD_byear_predict.csv', sep='\t', index=False)